In [2]:
#@title Global Vars
# Function to get user input for global variables
def get_global_vars():
    today_date_str = input("Enter today's date: ")
    expr_date_str = input("Enter the expiration date: ")

    time_frame_type = input("Enter the time frame type (Enter $ for dollar return or % for percentage change): ")
    time_frame_str = input("Enter the time frame (1 mo, YTD, TTM, 2023, 2022, 2021, All): ")
    
    base_dir = input("Enter the base directory")
    
    n = input("Enter n (for maxloss): ")

    time_frame_dict = {
        "$": {
            '1 mo': 10,
            'YTD': 11,
            'TTM': 12,
            '2023': 13,
            '2022': 14,
            '2021': 15,
            'All': 16
        },
        "%": {
            '1 mo': 19,
            'YTD': 20,
            'TTM': 21,
            '2023': 22,
            '2022': 23,
            '2021': 24,
            'All': 25
        }
    }

    time_frame = time_frame_dict[time_frame_type][time_frame_str]

    return today_date_str, expr_date_str, time_frame, base_dir, n

# Get user input for global variables
today_date_str, expr_date_str, time_frame, base_dir, n = get_global_vars()

print(f"Today's date: {today_date_str}")
print(f"Expiration date: {expr_date_str}")
print(f"Time frame: {time_frame}")
print(f"Base_dir: {base_dir}")
print(f"N: {n}")

Today's date: 06-13-2024
Expiration date: 2024-06-28
Time frame: 21
Base_dir: data/6-13


NameError: name 'n' is not defined

In [4]:
#@title Imports
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import zipfile
import os
from sklearn.model_selection import train_test_split
from skopt.space import Real, Categorical
from skopt.utils import use_named_args
from sklearn.model_selection import KFold
from skopt import gp_minimize
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment, PatternFill, Font
from openpyxl.utils.dataframe import dataframe_to_rows
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import PyPDF2
from collections import deque
from joblib import Parallel, delayed
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from itertools import product
import csv

In [5]:
main_data_file_path = f"{base_dir}/Main.csv"
extracted_folder = f"{base_dir}/Options"
# Check if the file exists
print("Main data file exists:", os.path.exists(main_data_file_path))
print("Extracted folder exists:", os.path.exists(extracted_folder))

Main data file exists: True
Extracted folder exists: True


In [6]:
#@title Setup
# Read the main data file
df = pd.read_csv(main_data_file_path)

# Get the list of stocks in the main CSV
csv_stocks = [df.columns[i + 2].split('_')[0] for i in range(1, len(df.columns) - 4, 3)]
print(f"Stocks in main CSV: {csv_stocks}")

# Get the list of stocks in the zip files
zip_stocks = set()
for filename in os.listdir(extracted_folder):
    stock_name = filename.split('_')[0]
    zip_stocks.add(stock_name)
zip_stocks = list(zip_stocks)
print(f"Stocks in zip files: {zip_stocks}")

# Find the intersection of stocks
all_stocks = list(set(csv_stocks) & set(zip_stocks))

# Filter out stocks that do not have files for today_date_str or expr_date_str
filtered_stocks = []
for stock in all_stocks:
    has_today_date = any(f"{stock}_{today_date_str}_" in filename for filename in os.listdir(extracted_folder))
    has_expr_date = any(f"{stock}_{today_date_str}_calls_exp_{expr_date_str}" in filename or f"{stock}_{today_date_str}_puts_exp_{expr_date_str}" in filename for filename in os.listdir(extracted_folder))
    if has_today_date and has_expr_date:
        filtered_stocks.append(stock)

all_stocks = sorted(filtered_stocks)
print(f"Combined list of stocks after filtering by dates: {all_stocks}")

# Function to remove rows with volume less than 10% of the volume closest to the current price
def filter_option_data_by_volume(extracted_folder, main_data_file_path, all_stocks):
    # Read the main data file
    main_df = pd.read_csv(main_data_file_path)

    for stock in all_stocks:
        # Get the current price for the stock
        current_price = main_df[f'{stock}_adjclose'].iloc[0]

        for filename in os.listdir(extracted_folder):
            if filename.startswith(f"{stock}_"):
                file_path = os.path.join(extracted_folder, filename)
                df = pd.read_csv(file_path)

                if 'strike' in df.columns and 'volume' in df.columns:

                    # Filter out rows with NaN volumes
                    df_filtered = df.dropna(subset=['volume'])

                    # If the DataFrame is empty after filtering, skip to the next file
                    if df_filtered.empty:
                        print(f"No rows with non-NaN volumes found for stock {stock}.")
                        continue

                    # Find the closest strike price to the current price
                    df_filtered = df_filtered.copy()  # Make a copy to avoid SettingWithCopyWarning
                    df_filtered['diff'] = abs(df_filtered['strike'] - current_price)
                    closest_strike_row = df_filtered.loc[df_filtered['diff'].idxmin()]
                    closest_strike_volume = closest_strike_row['volume']

                    # Calculate the volume threshold
                    volume_threshold = 0.1 * closest_strike_volume

                    # Filter the DataFrame
                    df_filtered = df_filtered[df_filtered['volume'] >= volume_threshold]

                    # Save the filtered DataFrame back to the file
                    df_filtered.to_csv(file_path, index=False)

    print("Option data filtered by volume successfully.")

# Filter the option data by volume
filter_option_data_by_volume(extracted_folder, main_data_file_path, all_stocks)

Stocks in main CSV: ['AAPL', 'ABNB', 'AFRM', 'ALGN', 'AMAT', 'AMD', 'AMGN', 'AMKR', 'AMP', 'APA', 'APP', 'AVGO', 'BCC', 'BLDR', 'BTC-USD', 'C', 'CAVA', 'CCL', 'CFLT', 'CI', 'CLF', 'COIN', 'CROX', 'CVNA', 'CVX', 'CZR', 'DASH', 'DDS', 'DFS', 'DUOL', 'ENPH', 'ENTG', 'ETH-USD', 'FCX', 'FND', 'GFS', 'GOOGL', 'GPS', 'GS', 'GTLB', 'HOOD', 'HUBS', 'INTC', 'IOT', 'JPM', 'KLAC', 'KO', 'LLY', 'LNW', 'LRCX', 'LSCC', 'MCHP', 'MDB', 'META', 'MGM', 'MKSI', 'MPWR', 'MRNA', 'MRVL', 'MS', 'MSFT', 'MSTR', 'MTDR', 'NFLX', 'NVO', 'ON', 'ONTO', 'OVV', 'OXY', 'PANW', 'PATH', 'PAYC', 'PCOR', 'PEP', 'PINS', 'PLTR', 'RBLX', 'RCL', 'RIVN', 'RKT', 'SNOW', 'SPY', 'SQ', 'TOST', 'TPR', 'TSLA', 'TTD', 'U', 'UNH', 'VRTX', 'W', 'WCC', 'WEX', 'XLC', 'XLF', 'XLK', 'XLY', 'XOM', 'XPO', 'DJI', 'IXIC', 'TYX']
Stocks in zip files: ['ON', 'MKSI', 'XPO', 'W', 'MSTR', 'SQ', 'TOST', 'CVX', 'MSFT', 'MPWR', 'GS', 'FCX', 'AFRM', 'GOOGL', 'WEX', 'CAVA', 'APP', 'CROX', 'MRVL', 'HUBS', 'PINS', 'AMP', 'DFS', 'DASH', 'ONTO', 'CCL', 'LRC

In [7]:
#@title Change days
# Function to calculate percent change for the specified day range
def calculate_percent_change(df, days):
    # Ensure the day range is valid
    if days <= 0:
        raise ValueError("Day range must be greater than 0.")

    # Iterate over each stock (assumed to be every 3 columns)
    numCols = len(df.columns) - 4
    for i in range(1, numCols, 3):
        signal_col = df.columns[i]  # Signal column
        pct_change_col = df.columns[i + 1]  # Percent change column
        adj_close_col = df.columns[i + 2]  # Adjusted close column

        # Initialize the pct change column
        df[pct_change_col] = [np.nan] * len(df)

        for j in range(len(df)):
            if j >= days:
                old_price = df.at[j, adj_close_col]
                new_price = df.at[j - days, adj_close_col]
                if old_price != 0:  # Avoid division by zero
                    pct_change = (new_price - old_price) / old_price * 100
                    df.at[j, pct_change_col] = pct_change
                else:
                    df.at[j, pct_change_col] = 0.0
    return df

# Function to calculate the number of weekdays between two dates
def calculate_weekdays(start_date_str, end_date_str):
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)
    weekdays = pd.date_range(start=start_date, end=end_date, freq='B')
    return len(weekdays)-1

# Read the main data file
df = pd.read_csv(main_data_file_path)

# Calculate the number of weekdays between today_date_str and expr_date_str
days = calculate_weekdays(today_date_str, expr_date_str)

# Calculate percent change
df = calculate_percent_change(df, days)

print(days)

# Save the updated DataFrame to a new CSV file
main_data_file_path = f'{base_dir}/updated_main_data.csv'
df.to_csv(main_data_file_path, index=False)

print(f"Updated CSV saved to {main_data_file_path}")


6
Updated CSV saved to data/6-13/updated_main_data.csv


In [8]:
#@title get_df
def get_percents(ante_price, ticker, time_frame, dummy_df, all_days, call_buy=None, call_sell=None, put_sell=None, put_buy=None, debug=False):
    # Define a function to parse dates
    def parse_dates(date):
        for fmt in ("%Y-%m-%d", "%m/%d/%y"):
            try:
                return pd.to_datetime(date, format=fmt)
            except ValueError:
                continue
        # If no format matches, raise an error
        raise ValueError(f"Date {date} is not in a recognized format")

    # Read the main data file
    df = pd.read_csv(main_data_file_path)

    # Parse the dates
    df['date'] = df['date'].apply(parse_dates)

    # Get the current date
    current_date = df['date'].iloc[0]

    # Filter data according to the time frame
    if time_frame == '1 Mo':
        start_date = current_date - pd.DateOffset(months=1)
    elif time_frame == 'YTD':
        start_date = pd.Timestamp(year=current_date.year, month=1, day=1)
    elif time_frame == 'TTM':
        start_date = current_date - pd.DateOffset(years=1)
    elif time_frame == '2023':
        start_date = pd.Timestamp(year=2023, month=1, day=1)
        end_date = pd.Timestamp(year=2023, month=12, day=31)
    elif time_frame == '2022':
        start_date = pd.Timestamp(year=2022, month=1, day=1)
        end_date = pd.Timestamp(year=2022, month=12, day=31)
    elif time_frame == '2021':
        start_date = pd.Timestamp(year=2021, month=1, day=1)
        end_date = pd.Timestamp(year=2021, month=12, day=31)
    elif time_frame == 'All':
        start_date = df['date'].min()
    else:
        raise ValueError("Invalid time frame specified")

    # Apply the date filter
    if time_frame in ['2023', '2022', '2021']:
        df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    else:
        df = df[df['date'] >= start_date]

    # Get the current price
    current_price = df[f'{ticker}_adjclose'].iloc[-1]

    # Initialize variables
    win = 0.0
    loss = 0.0
    max_loss = 0.0
    ante = 0.0

    win_dollar = 0.0
    loss_dollar = 0.0
    max_loss_dollar = 0.0
    ante_dollar = 0.0

    win_count = 0
    loss_count = 0
    max_loss_count = 0
    if dummy_df.iloc[5, 2] == 0:
        return win, loss, max_loss, ante, win_dollar, loss_dollar, max_loss_dollar, ante_dollar, win_count, loss_count, max_loss_count

    # Adjust for positive ante
    if dummy_df.iloc[5, 1] > 0:
        numerator = ante_price
    else:
        numerator = -ante_price

    for idx, row in df.iterrows():
        if all_days or row[f'{ticker}_signal_exists'] == 1:
            pct_chg = row[f'{ticker}_5_pctchg'] / 100
            if debug: print("Before divide to get conversion rate")
            conversion_rate = numerator / (float(dummy_df.iloc[5, 1]) * row[f'{ticker}_adjclose'])
            if debug: print("After divide to get conversion rate")
            total = 0.0
            #Call buy
            if call_buy is not None:
                total += max(pct_chg-call_buy,0)
            #Call sell
            if call_sell is not None:
                total -= max(pct_chg-call_sell,0)
            #Put buy
            if put_buy is not None:
                total += max(put_buy-pct_chg,0)
            #Put sell
            if put_sell is not None:
                total -= max(put_sell-pct_chg,0)

            if total > 0:
                win += total
                win_dollar += total*row[f'{ticker}_adjclose'] * conversion_rate
                win_count += 1

            if total < 0:
                loss += total
                loss_dollar += total*row[f'{ticker}_adjclose'] * conversion_rate
                loss_count += 1

            ante += 1
            ante_dollar += float(dummy_df.iloc[5, 2]) * row[f'{ticker}_adjclose']

    return win, loss, max_loss, ante, win_dollar, loss_dollar, max_loss_dollar, ante_dollar, win_count, loss_count, max_loss_count

def find_closest_strike(ticker, target_price, option_type, current_price, debug=False):
    closest_strike = None
    closest_diff = float('inf')
    average_bid_ask = None
    option_type_alternatives = [option_type, option_type.lower() + 's']  # E.g., 'CALL' and 'calls' or 'PUT' and 'puts'

    if debug: print(f"Finding closest strike for {ticker}")
    for filename in os.listdir(extracted_folder):
        if any(f"{ticker}_{today_date_str}_{alt}_exp_{expr_date_str}" in filename for alt in option_type_alternatives):
            if debug: print("Found the option file")
            file_path = os.path.join(extracted_folder, filename)
            df = pd.read_csv(file_path)
            if 'strike' in df.columns and 'bid' in df.columns and 'ask' in df.columns:
                if debug: print(f"Columns are present for {ticker}!")
                df['diff'] = abs(df['strike'] - target_price)
                min_diff_row = df.loc[df['diff'].idxmin()]
                if min_diff_row['diff'] < closest_diff:
                    closest_diff = min_diff_row['diff']
                    closest_strike = min_diff_row['strike']
                    if debug: print("Before divide in strike price average")
                    average_bid_ask = (min_diff_row['bid'] + min_diff_row['ask']) / 2
                    if debug: print("After divide in strike price average")

    return closest_strike, average_bid_ask


def get_df(ticker, call_buy=None, call_sell=None, put_sell=None, put_buy=None, all_days=False, debug=False):
    # Read the main data file
    df = pd.read_csv(main_data_file_path)

    # Get the current price from the most recent price in the [ticker]_adjclose column
    current_price = df[f'{ticker}_adjclose'].iloc[0]

    # Step 1: Define the headers
    headers = [f'{ticker} Stock', 'Strike', '% change', '5 day option price', 'Column 5', 'Column 6']

    # Step 2: Define the first column data
    first_column_data = [
        ["Current price"],
        ["Call Buy"],
        ["Call Sell"],
        ["Put 1 (sell)"],
        ["Put 2 (buy)"],
        ["Net Ante"],
        [""],
        [""],
        ["Current Option Price"],
        ["$100 Ante"],
        ["1 Mo"],
        ["YTD"],
        ["TTM"],
        ["2023"],
        ["2022"],
        ["2021"],
        ["All"],
        [""],
        ["Percent Change"],
        ["1 Mo"],
        ["YTD"],
        ["TTM"],
        ["2023"],
        ["2022"],
        ["2021"],
        ["All"],
        [""],
        ["Win Rate"],
        ["1 Mo"],
        ["YTD"],
        ["TTM"],
        ["2023"],
        ["2022"],
        ["2021"],
        ["All"]
    ]

    # Convert to DataFrame
    first_column_df = pd.DataFrame(first_column_data, columns=[f'{ticker} Stock'])

    # Step 3: Create dummy data for the remaining columns
    num_rows = len(first_column_data)
    dummy_data = {
        'Strike': [''] * num_rows,
        '% change': [''] * num_rows,
        '5 day option price': [''] * num_rows,
        'Column 5': [''] * num_rows,
        'Column 6': [''] * num_rows
    }

    # Find the closest strikes and average bid/ask for the given targets
    call_buy_target_price = current_price * (1 + call_buy) if call_buy is not None else None
    closest_call_buy_strike, average_call_buy_bid_ask = find_closest_strike(ticker, call_buy_target_price, 'CALL', current_price, debug) if call_buy is not None else (None, None)

    call_sell_target_price = current_price * (1 + call_sell) if call_sell is not None else None
    closest_call_sell_strike, average_call_sell_bid_ask = find_closest_strike(ticker, call_sell_target_price, 'CALL', current_price, debug) if call_sell is not None else (None, None)

    putsell_target_price = current_price * (1 + put_sell) if put_sell is not None else None
    closest_putsell_strike, average_putsell_bid_ask = find_closest_strike(ticker, putsell_target_price, 'PUT', current_price, debug) if put_sell is not None else (None, None)

    putbuy_target_price = current_price * (1 + put_buy) if put_buy is not None else None
    closest_putbuy_strike, average_putbuy_bid_ask = find_closest_strike(ticker, putbuy_target_price,'PUT', current_price, debug) if put_buy is not None else (None, None)

    # Convert dummy data to DataFrame
    dummy_df = pd.DataFrame(dummy_data)
    dummy_df.at[0, 'Strike'] = current_price

    # Assign values to specific cells
    if call_buy is not None:
        dummy_df.at[1, 'Strike'] = closest_call_buy_strike
        dummy_df.at[1, '5 day option price'] = average_call_buy_bid_ask
        if debug: print("Before divide strike price for stock: "+ticker)
        dummy_df.at[1, '% change'] = closest_call_buy_strike / current_price - 1
        if debug: print("After divide strike price")

    if call_sell is not None:
        dummy_df.at[2, 'Strike'] = closest_call_sell_strike
        dummy_df.at[2, '5 day option price'] = average_call_sell_bid_ask
        dummy_df.at[2, '% change'] = closest_call_sell_strike / current_price - 1

    if put_sell is not None:
        dummy_df.at[3, 'Strike'] = closest_putsell_strike
        dummy_df.at[3, '5 day option price'] = average_putsell_bid_ask
        dummy_df.at[3, '% change'] = closest_putsell_strike / current_price - 1

    if put_buy is not None:
        dummy_df.at[4, 'Strike'] = closest_putbuy_strike
        dummy_df.at[4, '5 day option price'] = average_putbuy_bid_ask
        dummy_df.at[4, '% change'] = closest_putbuy_strike / current_price - 1

    # Net ante
    dummy_df.at[5, '5 day option price'] = (
        (float(dummy_df.at[3, '5 day option price']) if put_sell is not None and dummy_df.at[3, '5 day option price'] else 0)
        - (float(dummy_df.at[1, '5 day option price']) if call_buy is not None and dummy_df.at[1, '5 day option price'] else 0)
        - (float(dummy_df.at[4, '5 day option price']) if put_buy is not None and dummy_df.at[4, '5 day option price'] else 0)
        + (float(dummy_df.at[2, '5 day option price']) if call_sell is not None and dummy_df.at[2, '5 day option price'] else 0)
    )
    dummy_df.at[5, '% change'] = float(dummy_df.at[5, '5 day option price']) / float(dummy_df.at[0, 'Strike']) if dummy_df.at[0, 'Strike'] else 0.0

    # Define the time frames to process
    time_frames = ['1 Mo', 'YTD', 'TTM', '2023', '2022', '2021', 'All']
    row_mapping = {
        '1 Mo': 19,
        'YTD': 20,
        'TTM': 21,
        '2023': 22,
        '2022': 23,
        '2021': 24,
        'All': 25
    }

    row_mapping_dollar = {
        '1 Mo': 10,
        'YTD': 11,
        'TTM': 12,
        '2023': 13,
        '2022': 14,
        '2021': 15,
        'All': 16
    }

    row_mapping_rate = {
        '1 Mo': 28,
        'YTD': 29,
        'TTM': 30,
        '2023': 31,
        '2022': 32,
        '2021': 33,
        'All': 34
    }

    ante_price = 100
    flag = True
    while flag:
        flag = False
        for time_frame in time_frames:
            win, loss, max_loss, ante, win_dollar, loss_dollar, max_loss_dollar, ante_dollar, win_count, loss_count, max_loss_count = get_percents(ante_price, ticker, time_frame, dummy_df, all_days, call_buy, call_sell, put_sell, put_buy)
            if np.abs(max_loss_dollar) > 3 * np.abs(ante_dollar):
                flag = True
                break
            row = row_mapping[time_frame]
            dummy_df.at[row, 'Strike'] = win
            dummy_df.at[row, '% change'] = loss
            dummy_df.at[row, '5 day option price'] = max_loss
            dummy_df.at[row, 'Column 5'] = ante

            row_dollar = row_mapping_dollar[time_frame]
            dummy_df.at[row_dollar, 'Strike'] = win_dollar
            dummy_df.at[row_dollar, '% change'] = loss_dollar
            dummy_df.at[row_dollar, '5 day option price'] = max_loss_dollar
            dummy_df.at[row_dollar, 'Column 5'] = ante_dollar

            row_rate = row_mapping_rate[time_frame]
            total = win_count + loss_count + max_loss_count
            if total != 0:
                dummy_df.at[row_rate, 'Strike'] = win_count / total
                dummy_df.at[row_rate, '% change'] = loss_count / total
                dummy_df.at[row_rate, '5 day option price'] = max_loss_count / total

        if flag:
            ante_price = ante_dollar / (3 * max_loss_dollar)

    # Get percent totals
    for i in range(19, 26):
        total = 0
        for j in range(0, 4):
            try:
                if j == 3:
                    total += float(dummy_df.iloc[i, j]) * dummy_df.iloc[5, 1]
                else:
                    total += float(dummy_df.iloc[i, j])
            except ValueError:
                continue  # Skip non-numeric values
        dummy_df.at[i, 'Column 6'] = total

    # $100 Ante
    for i in range(10, 17):
        if dummy_df.iloc[5, 1] > 0:
            dummy_df.at[i, 'Column 5'] = 100 * dummy_df.at[i + 9, 'Column 5']
        else:
            dummy_df.at[i, 'Column 5'] = -100 * dummy_df.at[i + 9, 'Column 5']

    # Get price totals
    for i in range(10, 17):
        total = 0
        for j in range(0, 4):
            try:
                total += float(dummy_df.iloc[i, j])
            except ValueError:
                continue  # Skip non-numeric values
        dummy_df.at[i, 'Column 6'] = total

    dummy_df.iloc[9, 0] = 'Win $'
    dummy_df.iloc[9, 1] = 'Loss $'
    dummy_df.iloc[9, 2] = 'Max Loss $'
    dummy_df.iloc[9, 3] = 'Ante $'
    dummy_df.iloc[9, 4] = 'Net $'

    dummy_df.iloc[18, 0] = 'Win %'
    dummy_df.iloc[18, 1] = 'Loss %'
    dummy_df.iloc[18, 2] = 'Max Loss %'
    dummy_df.iloc[18, 3] = 'Ante #'
    dummy_df.iloc[18, 4] = 'Net %'

    dummy_df.iloc[27, 0] = 'Win %'
    dummy_df.iloc[27, 1] = 'Loss %'
    dummy_df.iloc[27, 2] = 'Max Loss %'

    # Step 4: Combine the first column DataFrame with the dummy data DataFrame
    combined_df = pd.concat([first_column_df, dummy_df], axis=1)

    # Add headers to the combined DataFrame
    combined_df.columns = headers

    return combined_df

In [9]:
get_df("GTLB", call_buy=0.05, call_sell=0.03, put_sell=-0.02, put_buy=-0.03, all_days=False, debug=True)

Finding closest strike for GTLB
Found the option file
Columns are present for GTLB!
Before divide in strike price average
After divide in strike price average
Finding closest strike for GTLB
Found the option file
Columns are present for GTLB!
Before divide in strike price average
After divide in strike price average
Finding closest strike for GTLB
Found the option file
Columns are present for GTLB!
Before divide in strike price average
After divide in strike price average
Finding closest strike for GTLB
Found the option file
Columns are present for GTLB!
Before divide in strike price average
After divide in strike price average
Before divide strike price for stock: GTLB
After divide strike price


,GTLB Stock,Strike,% change,5 day option price,Column 5,Column 6
0,Current price,45.23,,,,
1,Call Buy,47.0,0.039133,0.4,,
2,Call Sell,47.0,0.039133,0.4,,
3,Put 1 (sell),44.0,-0.027194,0.425,,
4,Put 2 (buy),44.0,-0.027194,0.425,,
5,Net Ante,,0.0,0.0,,
6,,,,,,
7,,,,,,
8,Current Option Price,,,,,
9,$100 Ante,Win $,Loss $,Max Loss $,Ante $,Net $


In [231]:
#@title portfolio optimize
def portfolio_optimize(all_days):

    # Define the parameter space
    space = [
        Categorical([None] + list(np.linspace(0.005, 0.2, num=20)), name='call_buy'),
        Categorical([None] + list(np.linspace(-0.2, -0.01, num=20)), name='call_sell'),
        Categorical([None] + list(np.linspace(-0.1, -0.01, num=10)), name='put_sell'),
        Categorical([None] + list(np.linspace(-0.2, -0.01, num=20)), name='put_buy')
    ]

    def run_analysis(stocks, call_buy, call_sell, put_sell, put_buy, time_frame):
        gain_values_pos_ante = []
        gain_values_neg_ante = []

        # Define the function to process each stock
        def process_stock(stock):
            stock_df = get_df(stock, call_buy, call_sell, put_sell, put_buy, all_days)
            if stock_df is not None:
                try:
                    ante_value = stock_df.iloc[5, 2]
                    gain_value = stock_df.iloc[time_frame, 5]
                    # Check if the ante value is negative
                    if ante_value < 0:
                        return None, gain_value
                    else:
                        return gain_value, None
                except (IndexError, KeyError):
                    return None, None
            return None, None

        # Use parallel processing to process stocks
        results = Parallel(n_jobs=-1)(delayed(process_stock)(stock) for stock in stocks)
        #results = [process_stock(stock) for stock in stocks]

        # Separate positive and negative ante results
        for gain_pos, gain_neg in results:
            if gain_pos is not None:
                gain_values_pos_ante.append(gain_pos)
            if gain_neg is not None:
                gain_values_neg_ante.append(gain_neg)

        # Calculate the average gain value for positive ante
        if gain_values_pos_ante:
            average_gain_pos_ante = sum(gain_values_pos_ante) / len(gain_values_pos_ante)
        else:
            average_gain_pos_ante = None

        # Calculate the average gain value for negative ante
        if gain_values_neg_ante:
            average_gain_neg_ante = sum(gain_values_neg_ante) / len(gain_values_neg_ante)
        else:
            average_gain_neg_ante = None

        return average_gain_pos_ante, average_gain_neg_ante

    # Split the stocks into training and validation sets
    train_stocks, val_stocks = train_test_split(all_stocks, test_size=0.2, random_state=42)

    # Define the objective function for positive ante
    @use_named_args(space)
    def objective_pos_ante(**params):
        call_buy = params.get('call_buy', None)
        call_sell = params.get('call_sell', None)
        put_sell = params.get('put_sell', None)
        put_buy = params.get('put_buy', None)
        # Ensure valid combinations of options
        if call_buy is None or (call_sell is not None and call_sell >= call_buy) or put_buy is None:
            return 0.0

        gain_pos_ante, _ = run_analysis(train_stocks, call_buy, call_sell, put_sell, put_buy, time_frame)
        if not isinstance(gain_pos_ante, (int, float)) or np.isnan(gain_pos_ante):
            return 0.0
        return -gain_pos_ante  # Minimize negative gain to maximize gain

    # Define the objective function for negative ante
    @use_named_args(space)
    def objective_neg_ante(**params):
        call_buy = params.get('call_buy', None)
        call_sell = params.get('call_sell', None)
        put_sell = params.get('put_sell', None)
        put_buy = params.get('put_buy', None)
        # Ensure valid combinations of options
        if call_buy is None or (call_sell is not None and call_sell >= call_buy) or put_buy is None:
            return 0.0

        _, gain_neg_ante = run_analysis(train_stocks, call_buy, call_sell, put_sell, put_buy, time_frame)
        if not isinstance(gain_neg_ante, (int, float)) or np.isnan(gain_neg_ante):
            return 0.0
        return -gain_neg_ante  # Minimize negative gain to maximize gain

    # Perform Bayesian optimization for positive ante
    result_pos_ante = gp_minimize(objective_pos_ante, space, n_calls=30, random_state=0)
    print("Best parameters found for positive ante: call_buy={}, call_sell={}, put_sell={}, put_buy={}".format(result_pos_ante.x[0], result_pos_ante.x[1], result_pos_ante.x[2], result_pos_ante.x[3]))
    train_gain_pos_ante, _ = run_analysis(train_stocks, result_pos_ante.x[0], result_pos_ante.x[1], result_pos_ante.x[2], result_pos_ante.x[3], time_frame)
    print(f"Average gain after ante for training set with positive ante: {train_gain_pos_ante}")

    # Perform Bayesian optimization for negative ante
    result_neg_ante = gp_minimize(objective_neg_ante, space, n_calls=30, random_state=0)
    print("Best parameters found for negative ante: call_buy={}, call_sell={}, put_sell={}, put_buy={}".format(result_neg_ante.x[0], result_neg_ante.x[1], result_neg_ante.x[2], result_neg_ante.x[3]))
    _, train_gain_neg_ante = run_analysis(train_stocks, result_neg_ante.x[0], result_neg_ante.x[1], result_neg_ante.x[2], result_neg_ante.x[3], time_frame)
    print(f"Average gain after ante for training set with negative ante: {train_gain_neg_ante}")

    # Validate on the remaining 20% stocks
    val_gain_pos_ante = run_analysis(val_stocks, result_pos_ante.x[0], result_pos_ante.x[1], result_pos_ante.x[2], result_pos_ante.x[3], time_frame)
    val_gain_neg_ante = run_analysis(val_stocks, result_neg_ante.x[0], result_neg_ante.x[1], result_neg_ante.x[2], result_neg_ante.x[3], time_frame)
    print(f"Validation gain after ante for validation set with positive ante: {val_gain_pos_ante[0]}")
    print(f"Validation gain after ante for validation set with negative ante: {val_gain_neg_ante[1]}")

    return train_gain_pos_ante, train_gain_neg_ante, val_gain_pos_ante, val_gain_neg_ante, result_pos_ante.x[0], result_pos_ante.x[1], result_pos_ante.x[2], result_pos_ante.x[3], result_neg_ante.x[0], result_neg_ante.x[1], result_neg_ante.x[2], result_neg_ante.x[3]

In [436]:
#@title run portfolio optimize
train_gain_pos_ante, train_gain_neg_ante, val_gain_pos_ante, val_gain_neg_ante, poscallbuy, poscallsell, posputsell, posputbuy, negcallbuy, negcallsell, negputsell, negputbuy = portfolio_optimize(False)

Best parameters found for positive ante: call_buy=0.0768421052631579, call_sell=-0.04000000000000001, put_sell=-0.01, put_buy=None
Average gain after ante for training set with positive ante: -0.30788079160081316
Best parameters found for negative ante: call_buy=0.05631578947368421, call_sell=-0.01, put_sell=-0.1, put_buy=-0.03
Average gain after ante for training set with negative ante: 0.35690707968728225
Validation gain after ante for validation set with positive ante: -0.15862257358039047
Validation gain after ante for validation set with negative ante: None


In [218]:
#@title individual optimize
def individual_optimize(all_days):
    def split_stock_df(df, stock_name):
        signal_indices = df.index[df[f'{stock_name}_signal_exists'] == 1].tolist()
        num_signals = len(signal_indices)
        if num_signals < 10:
            return None, None

        split_index = int(0.8 * num_signals)
        train_indices = signal_indices[:split_index]
        val_indices = signal_indices[split_index:]

        train_df = df.loc[train_indices].reset_index(drop=True)
        val_df = df.loc[val_indices].reset_index(drop=True)

        return train_df, val_df

    def run_analysis(stock_name, call_buy, call_sell, put_sell, put_buy, time_frame, all_days):
        stock_df = get_df(stock_name, call_buy, call_sell, put_sell, put_buy, all_days=all_days)
        if stock_df is not None:
            gain_value = stock_df.iloc[time_frame, 5]
            return gain_value
        return np.nan

    def optimize_for_stock(stock_name, train_df, val_df, combination, all_days):
        # Define the parameter space for the specific combination
        space = []
        if 'call_buy' in combination:
            space.append(Categorical(list(np.linspace(0.005, 0.2, num=20)), name='call_buy'))
        if 'call_sell' in combination:
            space.append(Categorical(list(np.linspace(-0.2, -0.01, num=20)), name='call_sell'))
        if 'put_sell' in combination:
            space.append(Categorical(list(np.linspace(-0.1, -0.01, num=10)), name='put_sell'))
        if 'put_buy' in combination:
            space.append(Categorical(list(np.linspace(-0.2, -0.01, num=20)), name='put_buy'))

        @use_named_args(space)
        def objective(**params):
            call_buy = params.get('call_buy', None)
            call_sell = params.get('call_sell', None)
            put_sell = params.get('put_sell', None)
            put_buy = params.get('put_buy', None)

            # Enforce constraints
            if call_buy is None or (call_sell is not None and call_sell >= call_buy) or put_buy is None:
                return 0.0

            gain = run_analysis(stock_name, call_buy, call_sell, put_sell, put_buy, time_frame, all_days)
            if np.isnan(gain):
                return 0.0
            return -gain

        result = gp_minimize(objective, space, n_calls=10, random_state=42)

        # Map the result back to the appropriate parameters
        call_buy_opt = None
        call_sell_opt = None
        put_sell_opt = None
        put_buy_opt = None

        result_index = 0
        if 'call_buy' in combination:
            call_buy_opt = result.x[result_index]
            result_index += 1
        if 'call_sell' in combination:
            call_sell_opt = result.x[result_index]
            result_index += 1
        if 'put_sell' in combination:
            put_sell_opt = result.x[result_index]
            result_index += 1
        if 'put_buy' in combination:
            put_buy_opt = result.x[result_index]

        val_gain = run_analysis(stock_name, call_buy_opt, call_sell_opt, put_sell_opt, put_buy_opt, time_frame, all_days)
        if np.isnan(val_gain):
            val_gain = 0.0  # Ensure no NaN values
        return call_buy_opt, call_sell_opt, put_sell_opt, put_buy_opt, val_gain

    def process_stock(stock, all_days):
        train_df, val_df = split_stock_df(df, stock)
        if train_df is not None and val_df is not None:
            best_combination = None
            best_gain = -float('inf')
            best_params = None

            # Define the valid combinations
            valid_combinations = [
                ['call_buy', 'put_buy'],
                ['call_buy', 'call_sell', 'put_buy'],
                ['call_buy', 'put_sell', 'put_buy'],
                ['call_buy', 'call_sell', 'put_sell', 'put_buy']
            ]

            for combination in valid_combinations:
                params = optimize_for_stock(stock, train_df, val_df, combination, all_days)
                if params[4] > best_gain:
                    best_gain = params[4]
                    best_params = params
                    best_combination = combination

            return best_params
        return None, None, None, None, None

    optimized_params = {}
    validation_gains = []

    df = pd.read_csv(main_data_file_path)  # Load the main data file

    # Use parallel computing to optimize for each stock
    results = Parallel(n_jobs=-1)(delayed(process_stock)(stock, all_days) for stock in all_stocks)

    for stock, result in zip(all_stocks, results):
        if result is not None:
            call_buy_opt, call_sell_opt, put_sell_opt, put_buy_opt, val_gain = result
            if not np.isnan(val_gain):
                optimized_params[stock] = (call_buy_opt, call_sell_opt, put_sell_opt, put_buy_opt)
                validation_gains.append(val_gain)
            else:
                print(f"Stock {stock} returned a NaN validation gain")
        else:
            print(f"Stock {stock} optimization returned None")

    if validation_gains:
        average_validation_gain = sum(validation_gains) / len(validation_gains)
    else:
        average_validation_gain = np.nan

    print(f"Average gain after ante for validation sets across all stocks: {average_validation_gain}")

    return optimized_params, validation_gains, average_validation_gain

In [425]:
#@title run individual optimize
optimized_params, validation_gains, average_validation_gain = individual_optimize(all_days=False)

Average gain after ante for validation sets across all stocks: 0.3650599061332542


In [426]:
#@title debugging
# Find stocks that are in optimized_params.keys() but not in all_stocks
missing_stocks = set(optimized_params.keys()) - set(all_stocks)

# Print the missing stocks
if len(missing_stocks) == 0:
  print("All stocks accounted for")
else:
  print("Missing stocks:")
  for stock in missing_stocks:
      print(stock)
      
print(len(all_stocks))

All stocks accounted for
74


In [235]:
#@title Manual_optimize
def manual_optimize(all_days, n):
    # Define the parameter space with constraints
    def get_constrained_strikes(extracted_folder, ticker, option_type, current_price, max_steps=3):
        strikes = set()
        for filename in os.listdir(extracted_folder):
            parts = filename.split('_')
            if len(parts) >= 4 and parts[0] == ticker and parts[2] == option_type:
                file_path = os.path.join(extracted_folder, filename)
                df = pd.read_csv(file_path)
                if 'strike' in df.columns:
                    strikes.update(df['strike'].tolist())

        sorted_strikes = sorted(list(strikes))
        constrained_strikes = []

        for strike in sorted_strikes:
            if option_type == 'calls' and strike > current_price:
                constrained_strikes.append(strike)
            elif option_type == 'puts' and strike < current_price:
                constrained_strikes.append(strike)

            # Check if the number of steps is within the limit
            if len(constrained_strikes) >= max_steps:
                break

        return constrained_strikes

    # Get available strikes for each ticker with constraints
    def get_all_constrained_strikes(extracted_folder, all_stocks, main_data_file_path):
        main_df = pd.read_csv(main_data_file_path)
        available_strikes = {}
        for stock in all_stocks:
            current_price = main_df[f'{stock}_adjclose'].iloc[0]
            available_strikes[stock] = {
                'calls': get_constrained_strikes(extracted_folder, stock, 'calls', current_price),
                'puts': get_constrained_strikes(extracted_folder, stock, 'puts', current_price)
            }
        return available_strikes

    available_strikes = get_all_constrained_strikes(extracted_folder, all_stocks, main_data_file_path)

    def run_analysis(stock_name, call_buy, put_sell, put_buy, time_frame, all_days):
        stock_df = get_df(stock_name, call_buy, None, put_sell, put_buy, all_days=all_days)
        if stock_df is not None:
            gain_value = stock_df.iloc[time_frame, 5]
            ante = stock_df.iloc[5, 3]
            return gain_value, ante
        return np.nan, np.nan

    def optimize_for_stock(stock_name, current_price, train_df, val_df, all_days, n):
        # Get the available strikes for the stock with constraints
        call_strikes = available_strikes[stock_name]['calls']
        put_strikes = available_strikes[stock_name]['puts']

        best_params = None
        best_gain = -float('inf')

        # Try all combinations of strikes
        for call_buy, put_sell, put_buy in product(call_strikes, put_strikes, put_strikes):
            call_buy_pct = (call_buy / current_price) - 1
            put_sell_pct = (put_sell / current_price) - 1
            put_buy_pct = (put_buy / current_price) - 1

            # Ensure put buy is lower than put sell and constraints are met
            if (call_buy > current_price) and (put_sell < current_price) and (put_buy < current_price) and (put_buy < put_sell):
                gain, ante = run_analysis(stock_name, call_buy_pct, put_sell_pct, put_buy_pct, time_frame, all_days)
                max_loss = (put_sell-put_buy) + ante

                # Ensure max_loss is less than n times the ante
                if not np.isnan(ante) and max_loss < n * ante:
                    if not np.isnan(gain) and gain > best_gain:
                        best_gain = gain
                        best_params = (call_buy_pct, put_sell_pct, put_buy_pct)

        if best_params is not None:
            return *best_params, best_gain
        else:
            return None, None, None, best_gain


    def split_stock_df(df, stock_name):
        signal_indices = df.index[df[f'{stock_name}_signal_exists'] == 1].tolist()
        num_signals = len(signal_indices)
        if num_signals < 10:
            return None, None

        split_index = int(0.8 * num_signals)
        train_indices = signal_indices[:split_index]
        val_indices = signal_indices[split_index:]

        train_df = df.loc[train_indices].reset_index(drop=True)
        val_df = df.loc[val_indices].reset_index(drop=True)

        return train_df, val_df

    # Perform the optimization for each stock
    manual_params = {}
    manual_validation_gains = []

    df = pd.read_csv(main_data_file_path)  # Load the main data file

    for stock in all_stocks:
        current_price = df[f'{stock}_adjclose'].iloc[0]
        train_df, val_df = split_stock_df(df, stock)
        if train_df is not None and val_df is not None:
            result = optimize_for_stock(stock, current_price, train_df, val_df, all_days, n)
            if result is not None:
                call_buy_opt, put_sell_opt, put_buy_opt, val_gain = result
                if not np.isnan(val_gain):
                    manual_params[stock] = (call_buy_opt, put_sell_opt, put_buy_opt)
                    manual_validation_gains.append(val_gain)
                else:
                    print(f"Stock {stock} returned a NaN validation gain")
            else:
                print(f"Stock {stock} optimization returned None")
        else:
            print(f"Stock {stock} has insufficient data for optimization")

    if manual_validation_gains:
        average_manual_gain = sum(manual_validation_gains) / len(manual_validation_gains)
    else:
        average_manual_gain = np.nan

    print(f"Average %gain after ante for validation sets across all stocks: {average_manual_gain}")

    return manual_params, manual_validation_gains, average_manual_gain

In [428]:
#@title Run manual optimization
manual_params, manual_validation_gains, average_manual_gain = manual_optimize(all_days=False, n=n)
print(manual_validation_gains)

Average %gain after ante for validation sets across all stocks: 0.2093257128205095
[-0.04366411190979994, 0.029042562747342487, 0.6950295402896604, 0.17882229326998622, 0.030260055984186407, 0.11923909569498448, -0.017822776295649406, 0.09777250673332238, 0.11168986141155585, -0.1267537612745243, 0.11796979745585018, 0.19202892202845445, 0.013395152584799917, 0.12244646160405308, -0.013178870303646444, 1.6123411561734478, -0.08330413565348571, 1.725790000168268, -0.012315586914688904, -0.05723874688503244, 0.558887877258207, 0.548330767273788, -0.22826521704752378, 0.30376843909310375, 0.21986967437114682, 0.7568197651648585, -0.035257624501395746, -0.09373148987647206, 0.4907377797646342, 0.29961132785099576, -0.017851174627445926, 0.06461110726359913, 0.4273079325531395, 0.04975346232061794, 0.3608355376926612, 0.03091006429102866, -0.30135366942300296, 0.1909052987155673, 0.0023512769526997934, 1.4443212012387885, -0.03222067625660882, 0.039459068285548266, 0.13173552437576588, 1.23

In [429]:
#@save params
# Define the output CSV path
output_csv = os.path.join(base_dir, 'params.csv')

# Ensure the directory exists
os.makedirs(os.path.dirname(output_csv), exist_ok=True)

# Prepare data for writing
data = [
    ["Portfolio Optimize"],
    ["poscallbuy", poscallbuy],
    ["poscallsell", poscallsell],
    ["posputsell", posputsell],
    ["posputbuy", posputbuy],
    ["negcallbuy", negcallbuy],
    ["negcallsell", negcallsell],
    ["negputsell", negputsell],
    ["negputbuy", negputbuy],
    ["train_gain_pos_ante", train_gain_pos_ante],
    ["train_gain_neg_ante", train_gain_neg_ante],
    ["val_gain_pos_ante", val_gain_pos_ante],
    ["val_gain_neg_ante", val_gain_neg_ante],
    [],
    ["Individual Optimize"],
    ["optimized_params", str(optimized_params)],
    ["validation_gains", str(validation_gains)],
    ["average_validation_gain", average_validation_gain],
    [],
    ["Manual Optimize"],
    ["manual_params", str(manual_params)],
    ["manual_validation_gains", str(manual_validation_gains)],
    ["average_manual_gain", average_manual_gain]
]

# Write data to CSV
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"Results saved to {output_csv}")


Results saved to data/6-27/params.csv


In [311]:
#@title get_pdf
def get_pdf(individual_csv_dir, individual_pdf_dir, final_pdf_path, manual_params, optimized_params, poscallbuy, poscallsell, posputsell, posputbuy, negcallbuy, negcallsell, negputsell, negputbuy, all_days):
    # Define paths
    os.makedirs(individual_csv_dir, exist_ok=True)
    os.makedirs(individual_pdf_dir, exist_ok=True)

    # Read the main data file
    df = pd.read_csv(main_data_file_path)

    # Function to round numerical values in the DataFrame
    def round_df(df):
        for col in df.columns:
            df[col] = df[col].apply(lambda x: round(float(x), 3) if is_number(x) else x)

        # Round specific cells to nearest dollar or 2 decimals as specified
        dollar_columns = ['Strike', '% change','5 day option price', 'Column 5', 'Column 6']
        dollar_rows = range(10, 17)
        for col in dollar_columns:
            for row in dollar_rows:
                if col in df.columns and is_number(df.at[row, col]):
                    df.at[row, col] = round(float(df.at[row, col]))

        specific_cells = [
            (0, 'Strike'), (1, 'Strike'), (2, 'Strike'), (3, 'Strike'), (4, 'Strike'), (6, 'Strike'),
            (1, '5 day option price'), (2, '5 day option price'), (3, '5 day option price'), (4, '5 day option price'), (5,'5 day option price')
        ]
        for row, col in specific_cells:
            if col in df.columns and is_number(df.at[row, col]):
                df.at[row, col] = round(float(df.at[row, col]), 2)

        percent_cells = [
            (1, '% change'), (2, '% change'), (3, '% change'), (4, '% change'), (5, '% change'), (6, '% change')
        ]
        for row, col in percent_cells:
            if col in df.columns and is_number(df.at[row, col]):
                df.at[row, col] = f"{round(float(df.at[row, col]) * 100, 2)}%"

        percent_columns = ['Strike', '% change','5 day option price', 'Column 6']
        percent_rows = range(19,26)
        for col in percent_columns:
            for row in percent_rows:
                if col in df.columns and is_number(df.at[row, col]):
                    df.at[row, col] = f"{round(float(df.at[row, col]) * 100, 2)}%"

        percent_columns = ['Strike', '% change','5 day option price']
        percent_rows = range(28,35)
        for col in percent_columns:
            for row in percent_rows:
                if col in df.columns and is_number(df.at[row, col]):
                    df.at[row, col] = f"{round(float(df.at[row, col]) * 100, 2)}%"

        return df


    # Helper function to check if a value is a number
    def is_number(value):
        try:
            float(value)
            return True
        except (ValueError, TypeError):
            return False

    def add_label(df, label):
        # Create a label row
        label_row = pd.DataFrame([[label] + [''] * (df.shape[1] - 1)], columns=df.columns)
        # Concatenate the label row with the DataFrame
        labeled_df = pd.concat([label_row, df], ignore_index=True)
        return labeled_df


    def csv_to_pdf(csv_path, pdf_path, fontsize=10):
        df = pd.read_csv(csv_path)

        # Replace NaN values with empty strings
        df = df.fillna('')

        fig, ax = plt.subplots(figsize=(12, 8))  # Adjust size as needed
        ax.axis('tight')
        ax.axis('off')
        table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

        # Set font size for the table
        table.auto_set_font_size(False)
        table.set_fontsize(fontsize)

        # Highlight the top left cell if ante is positive
        if not df.iloc[6, 2].startswith('-'):
            cell = table[0, 0]
            cell.set_facecolor('green')
            cell.set_text_props(color='white')

        # Set specific width for the first column and enable text wrapping
        cell_dict = table.get_celld()
        for i in range(len(df.index) + 1):
            cell_dict[(i, 0)].set_width(0.25)
            cell_dict[(i, 0)].get_text().set_wrap(True)

        # Set borders and linewidth
        for key, cell in table.get_celld().items():
            cell.set_edgecolor('black')
            cell.set_linewidth(0.5)

        # Extract relevant values, check for None
        def parse_float(value):
            try:
                return float(value)
            except ValueError:
                return None

        pp = PdfPages(pdf_path)
        pp.savefig(fig, bbox_inches='tight')
        pp.close()
        plt.close(fig)



    def merge_pdfs(pdf_list, output_path):
        merger = PyPDF2.PdfMerger()
        for pdf in pdf_list:
            merger.append(pdf)
        merger.write(output_path)
        merger.close()

    pdf_files = []

    # Assuming all_stocks is already defined
    for stock in all_stocks:
        manual_df = get_df(stock, manual_params[stock][0], None, manual_params[stock][1], manual_params[stock][2], all_days)
        individual_df = get_df(stock, optimized_params[stock][0], optimized_params[stock][1], optimized_params[stock][2], optimized_params[stock][3], all_days)
        pos_portfolio_df = get_df(stock, poscallbuy, poscallsell, posputsell, posputbuy, all_days)
        neg_portfolio_df = get_df(stock, negcallbuy, negcallsell, negputsell, negputbuy, all_days)
        if pos_portfolio_df.iloc[time_frame,5] > neg_portfolio_df.iloc[time_frame,5]:
            portfolio_df = pos_portfolio_df
        else:
            portfolio_df = neg_portfolio_df

        manual_df = round_df(manual_df)
        individual_df = round_df(individual_df)
        portfolio_df = round_df(portfolio_df)

        # Add labels to each DataFrame
        manual_df = add_label(manual_df, 'Manual Selection')
        individual_df = add_label(individual_df, 'Learned Stock')
        portfolio_df = add_label(portfolio_df, 'Learned Portfolio')

        # Save to CSV
        csv_manual_path = os.path.join(individual_csv_dir, f'{stock}_manual_{today_date_str}_{expr_date_str}.csv')
        manual_df.to_csv(csv_manual_path, index=False)
        csv_individual_path = os.path.join(individual_csv_dir, f'{stock}_individual_{today_date_str}_{expr_date_str}.csv')
        individual_df.to_csv(csv_individual_path, index=False)
        csv_portfolio_path = os.path.join(individual_csv_dir, f'{stock}_portfolio_{today_date_str}_{expr_date_str}.csv')
        portfolio_df.to_csv(csv_portfolio_path, index=False)

        # Convert CSV to PDF
        pdf_manual_path = os.path.join(individual_pdf_dir, f'{stock}_manual_{today_date_str}_{expr_date_str}.pdf')
        csv_to_pdf(csv_manual_path, pdf_manual_path, fontsize=10)
        pdf_individual_path = os.path.join(individual_csv_dir, f'{stock}_individual_{today_date_str}_{expr_date_str}.pdf')
        csv_to_pdf(csv_individual_path, pdf_individual_path, fontsize=10)
        pdf_portfolio_path = os.path.join(individual_csv_dir, f'{stock}_portfolio_{today_date_str}_{expr_date_str}.pdf')
        csv_to_pdf(csv_portfolio_path, pdf_portfolio_path, fontsize=10)

        pdf_files.append(pdf_manual_path)
        pdf_files.append(pdf_individual_path)
        pdf_files.append(pdf_portfolio_path)

    # Merge all PDFs into one
    merge_pdfs(pdf_files, final_pdf_path)
    print(f"All PDFs merged into {final_pdf_path}")


In [430]:
#@title run get_pdf
# Define directories inside the current working directory
individual_csv_dir = os.path.join(base_dir, 'individual_csvs')
individual_pdf_dir = os.path.join(base_dir, 'individual_pdfs')
final_pdf_path = os.path.join(base_dir, f'SignalDays_{today_date_str}_{expr_date_str}.pdf')

get_pdf(individual_csv_dir, individual_pdf_dir, final_pdf_path, manual_params, optimized_params, poscallbuy, poscallsell, posputsell, posputbuy, negcallbuy, negcallsell, negputsell, negputbuy, all_days=False)

All PDFs merged into data/6-27/SignalDays_06-27-2024_2024-07-05.pdf


In [431]:
#@title Get trades

# Function to get user input for threshold
def get_return_threshold():
    while True:
        threshold_input = input("Enter the return threshold percentage (e.g., 10 for 10%) or press Enter to include all trades: ")
        if threshold_input == "":
            return None
        try:
            return float(threshold_input) / 100
        except ValueError:
            print("Invalid input. Please enter a number or press Enter to include all trades.")

# Get the return threshold
return_threshold = get_return_threshold()

# Assuming all_stocks and other necessary variables are already defined

# Initialize the final list of trades
final_trades = []

# Assuming all_stocks is already defined
for stock in all_stocks:

    # Generate DataFrames for the stock
    manual_df = get_df(stock, manual_params[stock][0], None, manual_params[stock][1], manual_params[stock][2])
    individual_df = get_df(stock, optimized_params[stock][0], optimized_params[stock][1], optimized_params[stock][2], optimized_params[stock][3])
    pos_portfolio_df = get_df(stock, poscallbuy, poscallsell, posputsell, posputbuy)
    neg_portfolio_df = get_df(stock, negcallbuy, negcallsell, negputsell, negputbuy)

    if pos_portfolio_df.iloc[time_frame, 5] > neg_portfolio_df.iloc[time_frame, 5]:
        portfolio_df = pos_portfolio_df
    else:
        portfolio_df = neg_portfolio_df

    # Find the trades for the current stock
    trades = [
        ('Manual Selection', manual_df),
        ('Learned Stock', individual_df),
        ('Learned Portfolio', portfolio_df)
    ]

    for strategy_name, trade_df in trades:
        trade_return = trade_df.iloc[time_frame, 5]

        if return_threshold is None or trade_return > return_threshold:
            call_buy_strike = trade_df.iloc[1, 1]
            call_sell_strike = trade_df.iloc[2, 1]
            put_sell_strike = trade_df.iloc[3, 1]
            put_buy_strike = trade_df.iloc[4, 1]
            ante = trade_df.iloc[5, 3]
            projected_return = trade_return

            final_trades.append({
                'Stock': stock,
                'Strategy': strategy_name,
                'Call Buy Strike': call_buy_strike,
                'Call Sell Strike': call_sell_strike,
                'Put Sell Strike': put_sell_strike,
                'Put Buy Strike': put_buy_strike,
                'Ante': ante,
                'Projected Return': projected_return
            })

# Define the CSV file path
os.makedirs(base_dir, exist_ok=True)
csv_file_path = os.path.join(base_dir, 'final_trades.csv')

# Write final trades to a CSV file
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['Stock', 'Strategy', 'Call Buy Strike', 'Call Sell Strike', 'Put Sell Strike', 'Put Buy Strike', 'Ante', 'Projected Return'])
    writer.writeheader()
    for trade in final_trades:
        writer.writerow(trade)

print(f"Final trades saved to {csv_file_path}")

# Function to calculate the actual return based on provided prices
def calculate_actual_returns(trades_df, stock_prices):
    actual_returns = []
    current_prices = []

    for _, row in trades_df.iterrows():
        stock = row['Stock']
        strategy = row['Strategy']
        call_buy_strike = row['Call Buy Strike']
        call_sell_strike = row['Call Sell Strike']
        put_sell_strike = row['Put Sell Strike']
        put_buy_strike = row['Put Buy Strike']
        ante = row['Ante']

        stock_price = stock_prices.get(stock, None)
        current_prices.append(stock_price)

        if stock_price is None:
            actual_returns.append(None)
            continue

        # Calculate the return for each trade
        if stock_price > call_buy_strike:
            call_buy_return = stock_price - call_buy_strike
        else:
            call_buy_return = 0

        if stock_price > call_sell_strike:
            call_sell_return = stock_price - call_sell_strike
        else:
            call_sell_return = 0

        if stock_price < put_sell_strike:
            put_sell_return = put_sell_strike - stock_price
        else:
            put_sell_return = 0

        if stock_price < put_buy_strike:
            put_buy_return = put_buy_strike - stock_price
        else:
            put_buy_return = 0

        total_return_dollars = call_buy_return - call_sell_return - put_sell_return + put_buy_return + ante
        actual_returns.append(total_return_dollars)

    trades_df['Current Price'] = current_prices
    trades_df['Actual Return'] = actual_returns
    return trades_df

# Define the dictionary of stock prices
may_31_prices = {
    'AAPL': 226.34,
    'ABNB': 152.5,
    'AFRM': 30.21,
    'ALGN': 243.6,
    'AMAT': 242.59,
    'AMD': 171.9,
    'AMGN': 310.88,
    'APA': 29.44,
    'APP': 83.22,
    'AVGO': 1703.31,
    'C': 64.03,
    'CAVA': 94.99,
    'CCL': 17.21,
    'CI': 319.12,
    'CLF': 15.39,
    'COIN': 222.23,
    'CROX': 145.94,
    'CVNA': 128.72,
    'CVX': 156.42,
    'CZR': 39.74,
    'DASH': 110.96,
    'DFS': 128.89,
    'ENPH': 97.14,
    'FCX': 51.52,
    'GOOGL': 182.15,
    'GPS': 23.89,
    'GS': 452.32,
    'GTLB': 49.72,
    'HOOD': 22.71,
    'INTC': 30.97,
    'IOT': 33.7,
    'JPM': 202.26,
    'KLAC': 824.51,
    'KO': 63.65,
    'LLY': 905.38,
    'LRCX': 1064.85,
    'MDB': 249.96,
    'META': 504.22,
    'MGM': 44.44,
    'MRNA': 118.75,
    'MRVL': 69.9,
    'MS': 97.19,
    'MSFT': 446.95,
    'MSTR': 1377.48,
    'NFLX': 674.88,
    'NVDA': 125.83,
    'NVO': 142.74,
    'ON': 68.55,
    'OXY': 63.03,
    'PANW': 339.01,
    'PATH': 12.68,
    'PEP': 164.93,
    'PINS': 44.07,
    'PLTR': 25.33,
    'RBLX': 37.21,
    'RCL': 159.43,
    'RIVN': 13.42,
    'RKT': 13.7,
    'SNOW': 135.09,
    'SPY': 544.22,
    'SQ': 64.49,
    'TOST': 25.77,
    'TPR': 42.79,
    'TSLA': 197.88,
    'TTD': 97.67,
    'U': 16.26,
    'UNH': 509.26,
    'VRTX': 468.72,
    'W': 51.45,
    'XLC': 85.66,
    'XLF': 41.11,
    'XLK': 226.23,
    'XLY': 182.4,
    'XOM': 115.12
}

# Process the final trades CSV file
trades_df = pd.read_csv(csv_file_path)
trades_df = calculate_actual_returns(trades_df, may_31_prices)

# Print the sum of actual returns for each strategy
strategy_sums = trades_df.groupby('Strategy')['Actual Return'].sum()
print("Sum of Actual Returns for each Strategy:")
print(strategy_sums)

# Save the updated DataFrame to a new CSV file
new_file_path = os.path.join(base_dir, 'final_trades_with_actual_returns.csv')
trades_df.to_csv(new_file_path, index=False)

print(f"Updated CSV file with actual returns saved to {new_file_path}.")

Final trades saved to data/6-27/final_trades.csv
Sum of Actual Returns for each Strategy:
Strategy
Learned Portfolio     17.445
Learned Stock       -123.110
Manual Selection     -77.410
Name: Actual Return, dtype: float64
Updated CSV file with actual returns saved to data/6-27/final_trades_with_actual_returns.csv.
